In [1]:
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F

import pandas as pd
import numpy as np
import h5py

import matplotlib.pyplot as plt
from collections import OrderedDict

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
random_seed = 1
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

In [3]:
# The reynolds numbers from 10 to 90 are used for training 
# The model is evaluated on the last 10 reynolds numbers from 91 to 100

outputs = np.zeros((90-10+1, 368))
for Re in range(10,91,1):
    outputs[Re-10] = np.loadtxt('pinns/reduced_finetuned/weights/weights_'+str(Re)+'.txt')
    
# Inputs represent the reynolds numbers used for training
# Outputs are the outputs of the hyperLoRA which are the weights of the reduced PINN components (rwo vector and column vector)
inputs = np.arange(10,91,1)
inputs = np.reshape(inputs, (inputs.shape[0],1))
inputs = Variable(torch.from_numpy(inputs).float(), requires_grad=False).to(device)
outputs = Variable(torch.from_numpy(outputs).float(), requires_grad=False).to(device)
#print("inputs", inputs.shape)
#print("outputs", outputs.shape)

In [4]:
class HyperLoRA(nn.Module):
    def __init__(self):
        super(HyperLoRA, self).__init__()
        
        self.layers = nn.Sequential(nn.Linear(1,512),
                      nn.ReLU(),
                      nn.Linear(512,512),
                      nn.ReLU(),
                      nn.Linear(512,256),
                      nn.ReLU(),
                      nn.Linear(256,256),
                      nn.ReLU(),
                      nn.Linear(256,128),
                      nn.ReLU(),
                      nn.Linear(128,368))
        
    def forward(self, Re):        
        weights = self.layers(Re)
        return weights

In [5]:
hlora = HyperLoRA()
hlora = hlora.to(device)
mse_cost_function = nn.MSELoss() 
optimizer = torch.optim.Adam(hlora.parameters(), lr=1e-3)
scheduler = lr_scheduler.StepLR(optimizer, step_size=20000, gamma=0.1, last_epoch=-1, verbose=False)

In [6]:
iterations = 100000
for epoch in range(iterations):
    optimizer.zero_grad()
    
    hlora_out = hlora.forward(inputs)
    Loss = mse_cost_function(hlora_out, outputs)
    Loss.backward()
    optimizer.step()
    scheduler.step()

    with torch.autograd.no_grad():
        if epoch%1000 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print('Epoch %d, LR: %.4e, Loss: %.4e' % (epoch, current_lr, Loss))


Epoch 0, LR: 1.0000e-03, Loss: 6.8708e-01
Epoch 1000, LR: 1.0000e-03, Loss: 1.2385e-01
Epoch 2000, LR: 1.0000e-03, Loss: 6.3781e-02
Epoch 3000, LR: 1.0000e-03, Loss: 5.2478e-02
Epoch 4000, LR: 1.0000e-03, Loss: 5.2267e-02
Epoch 5000, LR: 1.0000e-03, Loss: 4.2854e-02
Epoch 6000, LR: 1.0000e-03, Loss: 4.6816e-02
Epoch 7000, LR: 1.0000e-03, Loss: 4.9659e-02
Epoch 8000, LR: 1.0000e-03, Loss: 3.7492e-02
Epoch 9000, LR: 1.0000e-03, Loss: 3.3112e-02
Epoch 10000, LR: 1.0000e-03, Loss: 3.6131e-02
Epoch 11000, LR: 1.0000e-03, Loss: 3.1065e-02
Epoch 12000, LR: 1.0000e-03, Loss: 2.5613e-02
Epoch 13000, LR: 1.0000e-03, Loss: 3.5260e-02
Epoch 14000, LR: 1.0000e-03, Loss: 3.2886e-02
Epoch 15000, LR: 1.0000e-03, Loss: 2.7507e-02
Epoch 16000, LR: 1.0000e-03, Loss: 2.5256e-02
Epoch 17000, LR: 1.0000e-03, Loss: 3.1221e-02
Epoch 18000, LR: 1.0000e-03, Loss: 2.7287e-02
Epoch 19000, LR: 1.0000e-03, Loss: 2.1886e-02
Epoch 20000, LR: 1.0000e-04, Loss: 2.5347e-02
Epoch 21000, LR: 1.0000e-04, Loss: 1.7714e-02
E

In [7]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
    
    def forward(self, weights, vectors, x):
        W = weights[0, :40].view(20, 2)
        A = vectors[0, :20].view(20,1)
        B = vectors[0, 20:22].view(1,2)
        cur_weight = torch.add(W, torch.mm(A,B))
        h = torch.tanh(F.linear(x, weight = cur_weight, bias = vectors[0, 22:42].view(20)))
        i = 60
        j = 42
        for _ in range(5):
            W = weights[0, i:i+400].view(20, 20)
            A = vectors[0, j:j+20].view(20,1)
            B = vectors[0, j+20:j+40].view(1,20)
            cur_weight = torch.add(W, torch.mm(A,B))
            h = torch.tanh(F.linear(h, weight = cur_weight, bias = vectors[0, j+40:j+60].view(20)))
            i += 420
            j+=60
        W = weights[0, i:i+60].view(3, 20)
        A = vectors[0, j:j+3].view(3,1)
        B = vectors[0, j+3:j+23].view(1,20)
        cur_weight = torch.add(W, torch.mm(A,B))
        h = F.linear(h, weight = cur_weight, bias = vectors[0, j+23:j+26].view(3))
        return h

In [8]:
pinn = PINN()

f = h5py.File('data/colocation_points.h5', 'r')
X_col = f['X_col']
Y_col = f['Y_col']
X_col = np.asarray(X_col)
Y_col = np.asarray(Y_col)
X_col = Variable(torch.from_numpy(X_col).float(), requires_grad=True).to(device)
Y_col = Variable(torch.from_numpy(Y_col).float(), requires_grad=True).to(device)
inputs = torch.cat([X_col, Y_col], axis=1)
f.close()

pretrained_weights = np.loadtxt('pinns/pretrained/weights/weights_'+str(Re)+'.txt')
pretrained_weights = torch.tensor(pretrained_weights, dtype=torch.float32).view(1, pretrained_weights.size)
#print("weights: ", pretrained_weights.shape)

for Re in range(91,101,1):
    Re_tensor = torch.tensor(Re, dtype=torch.float32).view(1,1)
    predicted_vectors = hlora.forward(Re_tensor)
    #print("vectors: ", predicted_vectors.shape)
    predicted_solution = pinn.forward(pretrained_weights, predicted_vectors, inputs)
    
    f = h5py.File('data/Analytical_Solutions/colocation/Re_{}.h5'.format(Re), 'r')
    S_col = f['S_col']
    S_col = np.asarray(S_col)
    analytical_solution = Variable(torch.from_numpy(S_col).float(), requires_grad=True).to(device)
    f.close()
    
    print("Re: ", Re)
    # Mean L2 Error
    error = torch.linalg.norm(analytical_solution-predicted_solution,2)
    print('Mean L2 Error: %.4e' % (error))
    # Relative L2 Error
    L2_error = torch.linalg.norm(analytical_solution-predicted_solution,2)/torch.linalg.norm(analytical_solution,2)
    print('Relative L2 Error: %.4e \n' % (L2_error))

Re:  91
Mean L2 Error: 2.7430e+02
Relative L2 Error: 4.5764e+00 

Re:  92
Mean L2 Error: 2.7299e+02
Relative L2 Error: 4.5540e+00 

Re:  93
Mean L2 Error: 2.7167e+02
Relative L2 Error: 4.5315e+00 

Re:  94
Mean L2 Error: 2.7034e+02
Relative L2 Error: 4.5087e+00 

Re:  95
Mean L2 Error: 2.6900e+02
Relative L2 Error: 4.4860e+00 

Re:  96
Mean L2 Error: 2.6777e+02
Relative L2 Error: 4.4650e+00 

Re:  97
Mean L2 Error: 2.6662e+02
Relative L2 Error: 4.4453e+00 

Re:  98
Mean L2 Error: 2.6557e+02
Relative L2 Error: 4.4273e+00 

Re:  99
Mean L2 Error: 2.6460e+02
Relative L2 Error: 4.4107e+00 

Re:  100
Mean L2 Error: 2.6371e+02
Relative L2 Error: 4.3954e+00 

